In [12]:
import numpy as np 
import matplotlib.pyplot as plt 
from math import exp as e
from pandas import DataFrame
import itertools

## Question 2
Consider a binomial tree with S0 = 16, u = 1.25, d = 0.75, and 3 periods (so a total of 8 scenarios). Interest rate is r = 0.05. We are interested in an Asian-type option that pays (1
compare the average asset price S1+S2+S3 3
3Y3 − S3)+ at T = 3, where Y3 = S1 + S2 + S3. Thus, to determine the payoff we to the terminal stock price S3 and collect the
difference. Determine the no-arbitrage price of this contingent claim today: V0. Also determine the replicating portfolios for each time step t = 0, 1, 2 along the scenario 

In [13]:
# generate non-combing binomial tree with only u, d as entries
def expand_grid(data_dict):
    rows = itertools.product(*data_dict.values())
    return DataFrame.from_records(rows,columns=data_dict.keys())
s0 = 16
u=1.25
d=0.75
w=expand_grid({"N" + str(x+1):[u,d] for x in range (3) })
w=w.values # w is matrix with u,d as elements

# generate binomial tree with prices
def stock_prices(s0):
    s = np.zeros((8,3))
    for i in range(1,3):
        s[:,[0]]=w[:,[0]]*s0
        s[:,[i]] =w[:,[i]]*s[:,[i-1]]
    return s

stock_prices(16)


array([[20.  , 25.  , 31.25],
       [20.  , 25.  , 18.75],
       [20.  , 15.  , 18.75],
       [20.  , 15.  , 11.25],
       [12.  , 15.  , 18.75],
       [12.  , 15.  , 11.25],
       [12.  ,  9.  , 11.25],
       [12.  ,  9.  ,  6.75]])

In [23]:
s0 = 16
u=1.25
d=0.75
r=0.05
q=(1+r-d)/(u-d)

w=expand_grid({"N" + str(x+1):[u,d] for x in range (3) })
w=w.values # w is matrix with u,d as elements
def stock_prices(s0):
    s = np.zeros((8,3))
    for i in range(1,3):
        s[:,[0]]=w[:,[0]]*s0
        s[:,[i]] =w[:,[i]]*s[:,[i-1]]
    return s
s= stock_prices(16)  #8*4 matrix

# to compute the final payoff, aka v3
Y3 = np.zeros(8)
payoff=np.zeros(8)
for i in range(len(Y3)):
    Y3[i] = sum(s[i])
    payoff[i] = max((1/3*Y3[i] -s[i,[-1]]),0)
payoff   


array([0.        , 2.5       , 0.        , 4.16666667, 0.        ,
       1.5       , 0.        , 2.5       ])

### To get $v_0$ using formula: $v_0 = \frac{E[v_3]}{(1+r)^3}$

In [15]:
v0 = (q**2*(1-q)*sum(payoff[[1,2,4]])+q*(1-q)**2*sum(payoff[[3,5,6]])+(1-q)**3*payoff[-1] )/(1+r)**3
v0

0.9191232048374901

### To get $v_0$ using backward recursive method,$v_n(s,y) = \frac{1}{1+r}(q*v_{n+1}(s*u,y+us) +(1-q)*v_{n+1}(sd,y+ds))$ this answer questions Question3(i)

### USE Backward induction to get $v_n$

In [16]:
# backward induction to get the payoff at eatch time period
v2 = np.zeros(4)
v1 = np.zeros(2)
for i in range(4):
    v2[i] = (q*payoff[2*i]+(1-q)*payoff[2*i+1])/(1+r)  
for i in range(2):
    v1[i] =(q*v2[2*i]+(1-q)*v2[2*i+1])/(1+r)  
v0=(q*v1[0]+(1-q)*v1[1])/(1+r)  
print("v0(16,16) is", v0,"which is the same as the above method")


data = {"s": [25,15,15,9], "y": [61,51,43,37],"v2":v2}
data_v1={"s":[20,12],"y":[36,28],"v1":v1}
print(DataFrame(data_v1))
DataFrame(data)

v0(16,16) is 0.9191232048374902 which is the same as the above method
    s   y        v1
0  20  36  1.148904
1  12  28  0.689342


,s,y,v2
0,25,61,0.952381
1,15,51,1.587302
2,15,43,0.571429
3,9,37,0.952381


In [17]:
#replicating portfolios t = 0, 1, 2 along the scenario w = THT
delta2_th = (payoff[4]-payoff[5])/(((d*u*u)-(d*u*d))*s0)
b2_th = v2[2]-delta2_th*s0*d*u
delta1_t = (v2[2] - v2[3])/((u-d)*d*s0)
b1_t = v1[1]-delta1_t*s0*d
delta0 = (v1[0] - v1[1])/((u-d)*s0)
b0 = v0 - delta0*s0
data={"t=2":[delta2_th,b2_th],"t=1":[delta1_t,b1_t],"t=0": [delta0,b0]}
DataFrame(data, index=["delta","B"])

,t=2,t=1,t=0
delta,-0.200000,-0.063492,5.744520e-02
B,3.571429,1.451247,-4.440892e-16


### Q3 (iii)
Formula for number of shares:
$\delta_n(s,y) = \frac{v_{n+1}(su,y+us)-v_{n+1}(ds,y+ds)}{(u-d)s}$

## Question4 Up and in Barrier call

In [18]:
# generate non-combining binomial tree

u=1.05
d=0.95

w=expand_grid({"N" + str(x+1):[u,d] for x in range (8) })

w=w.values # w is matrix with u,d as elements

# count Head for each brach of binomial tree
count_H = np.zeros(256)
for i in range(256):
    count_H[i]=w.tolist()[i].count(1.05)


In [19]:
# create a binomial tree matrix
s = np.zeros((256,8))
for i in range(1,8):

    s[:,[0]]=w[:,[0]]*10
    s[:,[i]] =w[:,[i]]*s[:,[i-1]]
s

array([[10.5       , 11.025     , 11.57625   , ..., 13.40095641,
        14.07100423, 14.77455444],
       [10.5       , 11.025     , 11.57625   , ..., 13.40095641,
        14.07100423, 13.36745402],
       [10.5       , 11.025     , 11.57625   , ..., 13.40095641,
        12.73090859, 13.36745402],
       ...,
       [ 9.5       ,  9.025     ,  8.57375   , ...,  7.35091891,
         7.71846485,  7.33254161],
       [ 9.5       ,  9.025     ,  8.57375   , ...,  7.35091891,
         6.98337296,  7.33254161],
       [ 9.5       ,  9.025     ,  8.57375   , ...,  7.35091891,
         6.98337296,  6.63420431]])

In [20]:
def barrier(K,B):
    max_s = np.zeros(256)
    price = 0
    count = 0
    qn_sum=0
    for i in range(256):
        max_s[i]=(max( s[i])) # find the max of each brach
      
        if max_s[i] > B:
            vn = max((s[i][-1] - K), 0 )   # find the payoff if knocks in
            qn=q**(count_H[i])*(1-q)**(8-count_H[i])  # get the risk-netural prob
            qn_sum +=qn                              # adding probability for each iteration
            price += (vn*qn)/(1+0.01)**8             # get the expected price for option
            if vn > 0:
                count +=1                            # count number of non-zero payoff
    return price,count,qn_sum
p=np.zeros(5)
c=np.zeros(5)
Q=np.zeros(5)

# for each barrier,caculate the expected option price, number of non-zero payofff, risk-netural prob
for b,i in zip(np.array([11,11.5,12,12.5,13]), range(5)):
    p[i]= barrier(11,b)[0]
    c[i] =barrier(11,b)[1]
    Q[i]= barrier(11,b)[2]
    
DataFrame({"B": [11,11.5,12,12.5,13],"price": p ,"# of nonzero payoff": c,"risk-neutral prob": Q })



,B,price,# of nonzero payoff,risk-neutral prob
0,11.0,0.465635,37.0,0.550771
1,11.5,0.465635,37.0,0.483494
2,12.0,0.465635,37.0,0.358525
3,12.5,0.314749,17.0,0.171072
4,13.0,0.261939,10.0,0.113841


## Question 5
<img src="hw1-5.png"/>

For a scenario w, if the unserlying reaches K=11 in a given period, payoff is $(S_T -K)^+$; if the underlying never reach K=11 durin g a given period, then the payoff will have an additional coupon payment 1.5 

## Question6 Hit-Miss option


In [24]:
u = 1.01
d = 0.99
p = 0.5

# generate non-combinng binomial tree 
w=expand_grid({"N" + str(x+1):[u,d] for x in range (4) })
w=w.values 
def stock_prices(s0):
    '''
    input inital stock price
    output a binomail tree 16*4 matrix
    '''
    s = np.zeros((16,4))
    for i in range(1,4):
        s[:,[0]]=w[:,[0]]*s0
        s[:,[i]] =w[:,[i]]*s[:,[i-1]]
    return s
    
# generate binomial tree with initial price 1.05    
stock_a = stock_prices(1.05)

# counting number of the hit box
count=0
for k in range(16):
    if any(stock_a[k,[i]] <=1.07 and stock_a[k,[i]]>=1.03 for i in range(1,4)):
            count += 1

# compute hit box price using risk netural pricing
hit_price = 12*p**4
hit_price

0.75

In [25]:
count=0
for k in range(16):
    if any(stock_a[k,[i]] <=1.055 and stock_a[k,[i]]>=1.045 for i in range(1,4)):
            count += 1
miss_count = 16-count
miss_price =miss_count*p**(4)
miss_price

0.375